In [3]:
%load_ext autoreload
%autoreload 2
%cd ..

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/teo/userdata/danbi_work


In [4]:
import torch
import torch.nn as nn


In [15]:
class MultiEmbedding(nn.Module):
  def __init__(self, vocab_sizes: dict, emb_size: int) -> None:
    super().__init__()
    self.layers = []
    self.emb_size = emb_size
    for vocab_size in vocab_sizes.values():
      self.layers.append(nn.Embedding(vocab_size, self.emb_size))
    self.layers = nn.ModuleList(self.layers)
  
  def forward(self, x):
    return torch.sum(torch.stack([module(x[..., i]) for i, module in enumerate(self.layers)], dim=-1), dim=-1)

multiemb = MultiEmbedding({'a': 10, 'b': 20}, 4)
dummy = torch.randint(0, 9, (17, 31, 2))